In [ ]:
import coloredlogs
import importlib
import logging
import serial
import time

try:
    import board  # resets interfaces
except Exception as e:
    print('Board import error', e)

# Raw FTDI access
from pyftdi.i2c import I2cController

# Sensors
from adafruit_scd4x import SCD4X, SCD4X_DEFAULT_ADDR
from ph4_sense.sensors.athx0 import ahtx0_factory
from ph4_sense.sensors.ccs811 import CCS811Custom, css811_factory
from ph4_sense.sensors.scd4x import scd4x_factory
from ph4_sense.sensors.sgp30 import sgp30_factory
from ph4_sense.sensors import sgp41 as m_sgp41
from ph4_sense.sensors.sps30 import sps30_factory
from ph4_sense_py.sensors.hdc1080_ada import HDC1080
from ph4_sense_py.ftdi import FT2232HI2C

coloredlogs.install(level=logging.INFO)



In [ ]:
#i2c = I2cController()
#i2c.configure('ftdi://ftdi:2232h/1')
#slave = i2c.get_port(0x42)
#slave.write([0x01, 0x02, 0x03])

# Use FTDI interface
i2c=FT2232HI2C(0, frequency=100000)

sgp30 = None
sgp41 = None
scd4x = None
ccs811 = None
hdc1080 = None
sps30 = None

In [ ]:
print(f"0x40 = {0x40} = HDC1080")
print(f"0x58 = {0x58} = SGP30")
print(f"0x59 = {0x59} = SGP41")
print(f"0x5a = {0x5a} = CSS811")
print(f"0x62 = {0x62} = SCD40")
print(f"0x69 = {0x69} = SPS30")


In [ ]:
print('Scan:', i2c.scan())

In [ ]:
buff = bytearray(4) 
print(i2c.readfrom_into(0x69, buff))

In [ ]:
scd4x = SCD4X(i2c)
scd4x.start_periodic_measurement()

In [ ]:
print(scd4x.CO2)

In [ ]:
sgp30 = sgp30_factory(i2c, measure_test=True, iaq_init=False)

In [ ]:
importlib.reload(m_sgp41)
sgp41 = m_sgp41.sgp41_factory(i2c, measure_test=True, iaq_init=False)

In [ ]:
sgp41.execute_conditioning()

In [ ]:
sgp41.measure_raw()

In [ ]:
ccs811 = css811_factory(i2c)

In [ ]:
hdc1080 = HDC1080(i2c)
print(hdc1080.measurements)

In [ ]:
sps30 = sps30_factory(i2c)

In [ ]:
while True:
    if hdc1080:
        print(hdc1080.measurements)
    if scd4x: 
        print(scd4x.CO2)
    if sgp30: 
        print(sgp30.co2eq_tvoc())
    if ccs811:
        print(ccs811.read_data())
    if sps30:
        print(sps30.read())
    if sgp41:
        print(sgp41.measure_raw())
    time.sleep(2)

In [ ]:
DUST='/dev/tty.usbserial-2120'

In [ ]:
from ph4_sense_py.sensors import sps30_uart_ada
importlib.reload(sps30_uart_ada)

In [ ]:
sps30=sps30_uart_ada.SPS30AdaUart(DUST)

In [ ]:
sps30.read_firmware_version()

In [ ]:
sps30.read_values()

In [ ]:
from ph4_sense_py.sensors import zh03b_uart_ada
importlib.reload(zh03b_uart_ada)

In [ ]:
zh03 = zh03b_uart_ada.Zh03bUartAda(DUST)

In [ ]:
#print(zh03.dormant_mode(False))
print(zh03.set_qa())
#print(zh03.set_stream())
print(zh03.qa_read_sample())

In [ ]:
#zh03.qa_read_sample()

In [ ]:
zh03.read_sample()

In [ ]:
zh03.set_stream()

In [ ]:
while True:
    #print(zh03.qa_read_sample())
    #time.sleep(0.2)
    print(time.time(), zh03.read_sample())
    time.sleep(1)

In [ ]:
for i in range(100):
    zh03.uart.flush_output()
    zh03.dormant_mode(True)                

In [ ]:
sps30 = SPS30_I2C(i2c)

In [ ]:
while True:
    print(sps30.read())
    time.sleep(1)